In [ ]:
!nvidia-smi

Sun Jul 20 12:54:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   76C    P0             31W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate

In [ ]:
!pip uninstall -y transformers accelerate

Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0


In [ ]:
!pip install transformers accelerate

  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
  Using cached accelerate-1.9.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.53.2-py3-none-any.whl (10.8 MB)
Using cached accelerate-1.9.0-py3-none-any.whl (367 kB)


In [ ]:
!pip install --upgrade transformers accelerate peft

In [ ]:
!pip install evaluate --quiet

In [ ]:
from transformers import pipeline ,set_seed
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import evaluate
import matplotlib.pyplot as plt
import pandas as pd
import torch
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import accelerate
device="cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_ckpt="google/pegasus-cnn_dailymail"
tokenizer=AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!wget summarizer-data.zip

--2025-07-20 12:56:01--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2025-07-20 12:56:01--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.5’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.02s   

2025-07-20 12:56:02 (309 MB/s) - ‘summarizer-data.zip.5’ saved [7903

In [ ]:
!unzip summarizer-data.zip -d summarizer-data

Archive:  summarizer-data.zip
replace summarizer-data/samsum-test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: summarizer-data/samsum-test.csv  
replace summarizer-data/samsum-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: summarizer-data/samsum-train.csv  
replace summarizer-data/samsum-validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: summarizer-data/samsum-validation.csv  
replace summarizer-data/samsum_dataset/dataset_dict.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: summarizer-data/samsum_dataset/dataset_dict.json  
replace summarizer-data/samsum_dataset/test/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: summarizer-data/samsum_dataset/test/data-00000-of-00001.arrow  
replace summarizer-data/samsum_dataset/test/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: summarizer-data/samsum_dataset/test/dataset_info.json  
replace summarizer-data/samsum_dataset/test/state.

In [ ]:
from datasets import load_from_disk

In [ ]:
import os
os.path.exists("samsum_dataset")

True

In [ ]:
!pip install -U datasets gcsfs fsspec --quiet


In [ ]:
dataset_samsum = load_from_disk("samsum_dataset")

In [ ]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
split_legths=[len(dataset_samsum[split])for split in dataset_samsum]

print(f"split lengths:{split_legths}")
print(
    f"Features:{dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][1]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][1]["summary"])

split lengths:[14732, 819, 818]
Features:['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [ ]:
def convert_examples_to_features(example_batch):
  input_encodings=tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch['summary'],max_length=128,truncation=True)
  return {
      'input_ids':input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings['input_ids']
  }

In [ ]:
dataset_samsum_pt=dataset_samsum.map(convert_examples_to_features,batched=True)

In [ ]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [ ]:
from transformers import DataCollatorForSeq2Seq
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

In [ ]:
!pip install --upgrade transformers --quiet

In [ ]:
import transformers
print(transformers.__version__)

4.53.2


In [ ]:
from transformers import TrainingArguments
import inspect
print(inspect.getfile(TrainingArguments))

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py


In [ ]:
from transformers import TrainingArguments

trainer_args = TrainingArguments(
    output_dir="pegasus_Samsum",
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1, # Reduced batch size
    per_device_eval_batch_size=1, # Reduced batch size
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=int(1e6),
    gradient_accumulation_steps=32, # Increased accumulation steps
    report_to="none", # Disable W&B logging
    gradient_checkpointing=True, # Enable gradient checkpointing
    fp16=True # Enable mixed precision training
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt["train"],
    eval_dataset=dataset_samsum_pt["validation"]
)

/tmp/ipython-input-27-3624876842.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer=Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt["test"],eval_dataset=dataset_samsum_pt["validation"])

/tmp/ipython-input-28-2338785785.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(model=model_pegasus,args=trainer_args,tokenizer=tokenizer,data_collator=seq2seq_data_collator,train_dataset=dataset_samsum_pt["test"],eval_dataset=dataset_samsum_pt["validation"])


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3685: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=26, training_loss=2.9884764597966123, metrics={'train_runtime': 333.7601, 'train_samples_per_second': 2.454, 'train_steps_per_second': 0.078, 'total_flos': 314017624350720.0, 'train_loss': 2.9884764597966123, 'epoch': 1.0})

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))
    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        decoded_summaries=[tokenizer.decode(s,skip_special_tokens=True,clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries=[d.replace(""," ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
    score = metric.compute()
    return score

In [ ]:
import evaluate
rouge_names=["rouge1","rouge2","rougeL","rougeLsum"]
rouge_metric=evaluate.load('rouge')

In [ ]:
score=calculate_metric_on_test_ds(dataset_samsum['test'][0:10],rouge_metric,trainer.model,tokenizer,batch_size=2,column_text='dialogue',column_summary='summary')
rouge_dict=dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict,index=[f'pegasus'])

Streaming output truncated to the last 5000 lines.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key_value=None`.
Caching is incompatible with gradient checkpointing in PegasusDecoderLayer. Setting `past_key

AttributeError: 'numpy.float64' object has no attribute 'mid'